# Prepare Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import json

import warnings
warnings.filterwarnings('ignore')

drive_path = '/content/drive/MyDrive/Kuliah/Tugas Akhir/Final Project Shared Folder'
base_url = "https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?CycleBeginYear=2017"
dataset_names = ['Demographics', 'Dietary', 'Examination', 'Laboratory', 'Questionnaire']

Mounted at /content/drive


# Ingest Data

In [ ]:
data_path = "Dataset/Data Versioning/Combined_All_V6.csv"

df_raw = pd.read_csv(os.path.join(drive_path, data_path), index_col = 0)
df_raw = df_raw.set_index('SEQN')
df_raw.head()

,Demog1_RIAGENDR,Demog1_RIDAGEYR,Demog1_RIDRETH3,Demog1_DMDMARTL,Demog1_DMDHHSIZ,Demog1_DMDFMSIZ,Demog1_DMDHHSZA,Demog1_DMDHHSZB,Demog1_DMDHHSZE,Demog1_DMDEDUC,...,Dieta1_DR1TPROT,Dieta1_DR1TCARB,Dieta1_DR1TSUGR,Dieta1_DR1TFIBE,Dieta1_DR1TTFAT,Dieta1_DR1TSFAT,Dieta1_DR1TMFAT,Dieta1_DR1TPFAT,Dieta1_DR1TCHOL,Dieta1_DR1TCALC
SEQN,,,,,,,,,,,,,,,,,,,,,
93703.0,2.0,2.0,6.0,5.0,5.0,5.0,3.000000e+00,5.397605e-79,5.397605e-79,1.0,...,46.795385,183.161302,93.418166,11.005325,49.559349,17.118651,16.306485,10.77397,162.313609,893.39645
93704.0,1.0,2.0,3.0,5.0,4.0,4.0,2.000000e+00,5.397605e-79,5.397605e-79,1.0,...,51.580000,160.460000,76.970000,5.900000,43.240000,11.372000,14.333000,12.50600,144.000000,700.00000
93705.0,2.0,66.0,4.0,3.0,1.0,1.0,5.397605e-79,5.397605e-79,1.000000e+00,2.0,...,20.010000,157.450000,91.550000,8.400000,56.980000,16.435000,16.432000,19.78600,14.000000,314.00000
93706.0,1.0,18.0,6.0,5.0,5.0,5.0,5.397605e-79,5.397605e-79,1.000000e+00,4.0,...,94.190000,89.820000,14.730000,7.100000,137.390000,35.169000,45.805000,49.87300,462.000000,869.00000
93707.0,1.0,13.0,7.0,5.0,7.0,7.0,5.397605e-79,3.000000e+00,5.397605e-79,1.0,...,59.480000,188.150000,84.220000,10.900000,89.180000,33.252000,33.712000,12.42400,585.000000,535.00000


In [ ]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9254 entries, 93703.0 to 102956.0
Data columns (total 91 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Demog1_RIAGENDR  9254 non-null   float64
 1   Demog1_RIDAGEYR  9254 non-null   float64
 2   Demog1_RIDRETH3  9254 non-null   float64
 3   Demog1_DMDMARTL  9254 non-null   float64
 4   Demog1_DMDHHSIZ  9254 non-null   float64
 5   Demog1_DMDFMSIZ  9254 non-null   float64
 6   Demog1_DMDHHSZA  9254 non-null   float64
 7   Demog1_DMDHHSZB  9254 non-null   float64
 8   Demog1_DMDHHSZE  9254 non-null   float64
 9   Demog1_DMDEDUC   9254 non-null   float64
 10  Demog1_INDIN2    9254 non-null   float64
 11  Labor1_LBXTC     9254 non-null   float64
 12  Labor1_LBDTCSI   9254 non-null   float64
 13  Labor2_URXVOL1   9254 non-null   float64
 14  Labor2_URDFLOW1  9254 non-null   float64
 15  Labor2_URDTIME1  9254 non-null   float64
 16  Exami1_BPXPLS    9254 non-null   float64
 17  Exami1_BP

# Clean Data

In [ ]:
df = df_raw.copy()

# ----------------------------------------------------
# Fill in cvd status for person with less then 20 years old
# ----------------------------------------------------

# conditions = (df['Quest16_MCQ160B'].isna()) | (df['Quest16_MCQ160C'].isna()) | (df['Quest16_MCQ160D'].isna()) | (df['Quest16_MCQ160E'].isna()) | (df['Quest16_MCQ160F'].isna())
# df.loc[conditions, ['Quest16_MCQ160B', 'Quest16_MCQ160C', 'Quest16_MCQ160D', 'Quest16_MCQ160D', 'Quest16_MCQ160E', 'Quest16_MCQ160F']] = 1.0

# df['Quest16_MCQ160B'] = df['Quest16_MCQ160B'].fillna(1.0)

# ----------------------------------------------------
# Fill in Asthma and others
# ----------------------------------------------------

df[['Quest16_MCQ010', 'Quest16_MCQ220', 'Quest16_MCQ300C', 'Quest16_MCQ300A', 'Quest16_MCQ366A', 'Quest16_MCQ366B']] = df.loc[:, ['Quest16_MCQ010', 'Quest16_MCQ220', 'Quest16_MCQ300C', 'Quest16_MCQ300A', 'Quest16_MCQ366A', 'Quest16_MCQ366B']].fillna(value=9)

# ----------------------------------------------------
# Drop Unnecessary Column
# ----------------------------------------------------

df.drop(['Quest16_MCQ092'], axis=1, inplace=True)

# ----------------------------------------------------
# Label of heart failure disease (dropped other than heart failure)
# ----------------------------------------------------

df.drop(['Quest16_MCQ160C', 'Quest16_MCQ160D', 'Quest16_MCQ160E', 'Quest16_MCQ160F'], axis=1, inplace=True)

df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5569 entries, 93705.0 to 102956.0
Data columns (total 86 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Demog1_RIAGENDR  5569 non-null   float64
 1   Demog1_RIDAGEYR  5569 non-null   float64
 2   Demog1_RIDRETH3  5569 non-null   float64
 3   Demog1_DMDMARTL  5569 non-null   float64
 4   Demog1_DMDHHSIZ  5569 non-null   float64
 5   Demog1_DMDFMSIZ  5569 non-null   float64
 6   Demog1_DMDHHSZA  5569 non-null   float64
 7   Demog1_DMDHHSZB  5569 non-null   float64
 8   Demog1_DMDHHSZE  5569 non-null   float64
 9   Demog1_DMDEDUC   5569 non-null   float64
 10  Demog1_INDIN2    5569 non-null   float64
 11  Labor1_LBXTC     5569 non-null   float64
 12  Labor1_LBDTCSI   5569 non-null   float64
 13  Labor2_URXVOL1   5569 non-null   float64
 14  Labor2_URDFLOW1  5569 non-null   float64
 15  Labor2_URDTIME1  5569 non-null   float64
 16  Exami1_BPXPLS    5569 non-null   float64
 17  Exami1_BP

# Rearrange Column

In [ ]:
var_mapping_path = 'Dataset/Variable Mapping V2.xlsx'

var_mapping = pd.read_excel(os.path.join(drive_path, var_mapping_path))
var_mapping['group'] = var_mapping['group'].replace({'Dieta2': 'Dieta1'})
var_mapping['model_var_name'] = var_mapping['group'] + "_" + var_mapping['variable']

var_mapping

,model_var_name,name,title,variable,desc,is_used,lifestyle,group
0,Demog1_RIAGENDR,Demographics,Demographic Variables and Sample Weights,RIAGENDR,Gender,True,False,Demog1
1,Demog1_RIDAGEYR,Demographics,Demographic Variables and Sample Weights,RIDAGEYR,Age In Years At Screening,True,False,Demog1
2,Demog1_RIDRETH3,Demographics,Demographic Variables and Sample Weights,RIDRETH3,Race/Hispanic Origin W/ Nh Asian,True,False,Demog1
3,Demog1_DMDEDUC3,Demographics,Demographic Variables and Sample Weights,DMDEDUC3,Education Level - Children/Youth 6-19,True,False,Demog1
4,Demog1_DMDEDUC2,Demographics,Demographic Variables and Sample Weights,DMDEDUC2,Education Level - Adults 20+,True,False,Demog1
...,...,...,...,...,...,...,...,...
148,Demog1_DMDEDUC,Demographic,Demographic Variables and Sample Weights,DMDEDUC,Education Level all age,True,False,Demog1
149,Demog1_INDIN2,Demographic,Demographic Variables and Sample Weights,INDIN2,Average income from household income & family ...,True,False,Demog1
150,Exami1_SysPulse,Examination,Blood Pressure,SysPulse,Systolic Pulse Combined,True,False,Exami1
151,Exami1_DiaPulse,Examination,Blood Pressure,DiaPulse,Diastolic Pulse Combined,True,False,Exami1


In [ ]:
model_var = pd.DataFrame({'model_var_name': df.columns.tolist()})

model_var = pd.merge(model_var, var_mapping, left_on=model_var['model_var_name'].str.lower(), right_on=var_mapping['model_var_name'].str.lower(), how='outer', indicator=True).query('_merge!="right_only"')

# To Check if all variable is tagged
# model_var[model_var['_merge'] == "left_only"]

model_var = model_var[['model_var_name_x', 'lifestyle']].rename(columns={'model_var_name_x': 'model_var_name'}).drop_duplicates().reset_index(drop=True)

model_var

,model_var_name,lifestyle
0,Demog1_RIAGENDR,False
1,Demog1_RIDAGEYR,False
2,Demog1_RIDRETH3,False
3,Demog1_DMDMARTL,False
4,Demog1_DMDHHSIZ,False
...,...,...
81,Dieta1_DR1TSFAT,True
82,Dieta1_DR1TMFAT,True
83,Dieta1_DR1TPFAT,True
84,Dieta1_DR1TCHOL,True


In [ ]:
label_cols = list([col for col in model_var['model_var_name'] if "Quest16_MCQ160B" in col])

lifestyle_cols = model_var.loc[model_var['lifestyle'] == True, 'model_var_name'].tolist()
charac_cols = model_var.loc[model_var['lifestyle'] == False, 'model_var_name'].tolist()

lifestyle_cols = list(set(lifestyle_cols) - set(label_cols))
charac_cols = list(set(charac_cols) - set(label_cols))

print("Overall Cols:", len(model_var['model_var_name']))
print("Lifestyle Cols:", len(lifestyle_cols))
print(lifestyle_cols)
print("Characteristics Cols:", len(charac_cols))
print(charac_cols)
print("Label Cols:", len(label_cols))
print(label_cols)


Overall Cols: 86
Lifestyle Cols: 28
['Dieta1_DRDINT', 'Dieta1_DR1TFIBE', 'Quest21_SLQ300', 'Quest19_PAD660', 'Quest19_PAQ635', 'Dieta1_DR1TCHOL', 'Quest19_PAQ655', 'Dieta1_DR1TSFAT', 'Dieta1_DR1TKCAL', 'Exami2_BMXBMI', 'Dieta1_DR1TSUGR', 'Quest21_SLQ320', 'Quest21_SLQ330', 'Quest19_PAD615', 'Quest21_SLD012', 'Quest21_SLD013', 'Dieta1_DR1DAY', 'Quest6_DED120', 'Quest19_PAQ610', 'Quest6_DED125', 'Dieta1_DR1TPROT', 'Quest19_PAQ640', 'Dieta1_DR1TPFAT', 'Dieta1_DR1TMFAT', 'Dieta1_DR1TCALC', 'Dieta1_DR1TCARB', 'Dieta1_DR1TTFAT', 'Quest19_PAD645']
Characteristics Cols: 57
['Exami1_BPXPLS', 'Demog1_RIDRETH3', 'Demog1_DMDHHSZA', 'Demog1_DMDHHSZE', 'Quest14_INQ020', 'Quest18_OCQ210', 'Demog1_INDIN2', 'Quest12_HEQ030', 'Quest22_SMQ900', 'Exami2_BMXHT', 'Quest3_CDQ009', 'Quest3_CDQ010', 'Exami2_BMXWT', 'Quest3_CDQ008', 'Quest20_PFQ061H', 'Quest7_DIQ010', 'Quest20_PFQ061B', 'Labor1_LBDTCSI', 'Quest17_DPQ040', 'Demog1_RIAGENDR', 'Labor2_URDTIME1', 'Quest22_SMQ890', 'Demog1_DMDMARTL', 'Exami1_BPXPULS

In [ ]:
rearranged_df = df.copy()

rearranged_df = rearranged_df[lifestyle_cols + charac_cols + label_cols]

rearranged_df

,Dieta1_DRDINT,Dieta1_DR1TFIBE,Quest21_SLQ300,Quest19_PAD660,Quest19_PAQ635,Dieta1_DR1TCHOL,Quest19_PAQ655,Dieta1_DR1TSFAT,Dieta1_DR1TKCAL,Exami2_BMXBMI,...,Quest1_ALQ111,Quest10_ECQ020,Quest16_MCQ220,Quest4_CBD121,Quest16_MCQ366A,Labor2_URDFLOW1,Demog1_DMDEDUC,Quest9_DLQ050,Quest20_PFQ061C,Quest16_MCQ160B
SEQN,,,,,,,,,,,,,,,,,,,,,
93705.0,2.00000,8.400000,1380.0,71.704698,2.0,14.00000,3.353333,16.435000,1202.00000,31.700000,...,1.0,9.0,2.0,7.500000e+01,2.0,1.20400,2.0,2.0,1.0,2.0
93708.0,2.00000,16.600000,1350.0,71.704698,2.0,71.00000,3.353333,17.446000,1251.00000,23.700000,...,2.0,9.0,2.0,5.397605e-79,2.0,0.50000,1.0,2.0,2.0,2.0
93709.0,1.86000,17.436000,1350.0,67.828947,2.0,346.32000,3.210526,27.547400,2001.84000,38.900000,...,9.0,9.0,2.0,4.000000e+01,2.0,0.10700,4.0,1.0,9.0,2.0
93711.0,2.00000,44.500000,1320.0,60.000000,1.0,546.00000,4.000000,41.268000,2840.00000,21.300000,...,1.0,9.0,2.0,8.570000e+02,2.0,0.60500,5.0,2.0,9.0,2.0
93713.0,2.00000,14.600000,1350.0,68.558824,1.0,176.00000,3.357664,36.172000,2040.00000,23.500000,...,1.0,9.0,2.0,4.000000e+01,2.0,0.70600,3.0,2.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102950.0,1.84375,17.047917,180.0,72.043478,2.0,269.90625,3.271605,25.209792,2016.65625,28.919231,...,9.0,9.0,2.0,2.140000e+02,1.0,0.98811,3.0,1.0,2.0,1.0
102952.0,2.00000,28.700000,1350.0,65.134021,2.0,56.00000,3.371134,7.652000,1080.00000,20.000000,...,1.0,9.0,2.0,1.200000e+02,2.0,0.52500,3.0,2.0,1.0,2.0
102953.0,1.00000,57.000000,1320.0,61.261719,2.0,306.00000,3.394531,25.831000,3072.00000,35.800000,...,1.0,9.0,2.0,2.000000e+02,1.0,0.52900,3.0,2.0,9.0,2.0


# Discretization

PENTING, JANGAN DIHAPUS

In [ ]:
discretized_df = rearranged_df.copy()

temp = discretized_df.copy()

variable_discrete_val = {}

for ls in lifestyle_cols:
    variable_discrete_val[ls] = {}

    # Dont discretize already discretized columns
    if ls == 'Dieta2_DR1DAY' or len(discretized_df[ls].value_counts().to_list()) <=5:
        for k in df[ls].unique():
            variable_discrete_val[ls][str(k)] = k
        continue

    print(f'Categorizing column: {ls}, with value: ', end=' ')
    try:
        temp[ls] = pd.qcut(discretized_df[ls].to_numpy(), q=4, duplicates='drop') # For discretization label
        discretized_df[ls] = pd.qcut(discretized_df[ls].to_numpy(), q=4, labels=False, duplicates='drop') # Discretized columns

        for k, v in zip(discretized_df[ls].unique(), temp[ls].unique()):
            variable_discrete_val[ls][str(k)]=str(v)

    except Exception as error:
        print(f'{ls} cannot be discretized')
        print(f"ERROR DUE TO {error}")
        continue

print(variable_discrete_val)

Categorizing column: Dieta1_DRDINT, with value:  Categorizing column: Dieta1_DR1TFIBE, with value:  Categorizing column: Quest21_SLQ300, with value:  Categorizing column: Quest19_PAD660, with value:  Categorizing column: Dieta1_DR1TCHOL, with value:  Categorizing column: Quest19_PAQ655, with value:  Categorizing column: Dieta1_DR1TSFAT, with value:  Categorizing column: Dieta1_DR1TKCAL, with value:  Categorizing column: Exami2_BMXBMI, with value:  Categorizing column: Dieta1_DR1TSUGR, with value:  Categorizing column: Quest21_SLQ320, with value:  Categorizing column: Quest21_SLQ330, with value:  Categorizing column: Quest19_PAD615, with value:  Categorizing column: Quest21_SLD012, with value:  Categorizing column: Quest21_SLD013, with value:  Categorizing column: Dieta1_DR1DAY, with value:  Categorizing column: Quest6_DED120, with value:  Categorizing column: Quest19_PAQ610, with value:  Categorizing column: Quest6_DED125, with value:  Categorizing column: Dieta1_DR1TPROT, with value: 

In [ ]:
with open('variable_discrete_value.json', 'w') as json_file:
    json.dump(variable_discrete_val, json_file)

In [ ]:
discretized_df.head()

,Dieta1_DRDINT,Dieta1_DR1TFIBE,Quest21_SLQ300,Quest19_PAD660,Quest19_PAQ635,Dieta1_DR1TCHOL,Quest19_PAQ655,Dieta1_DR1TSFAT,Dieta1_DR1TKCAL,Exami2_BMXBMI,...,Quest1_ALQ111,Quest10_ECQ020,Quest16_MCQ220,Quest4_CBD121,Quest16_MCQ366A,Labor2_URDFLOW1,Demog1_DMDEDUC,Quest9_DLQ050,Quest20_PFQ061C,Quest16_MCQ160B
SEQN,,,,,,,,,,,,,,,,,,,,,
93705.0,1,0,2,2,2.0,0,2,1,0,2,...,1.0,9.0,2.0,7.500000e+01,2.0,1.204,2.0,2.0,1.0,2.0
93708.0,1,2,2,2,2.0,0,2,1,0,0,...,2.0,9.0,2.0,5.397605e-79,2.0,0.500,1.0,2.0,2.0,2.0
93709.0,0,2,2,1,2.0,2,1,2,1,3,...,9.0,9.0,2.0,4.000000e+01,2.0,0.107,4.0,1.0,9.0,2.0
93711.0,1,3,1,0,1.0,3,3,3,3,0,...,1.0,9.0,2.0,8.570000e+02,2.0,0.605,5.0,2.0,9.0,2.0
93713.0,1,1,2,1,1.0,1,2,3,2,0,...,1.0,9.0,2.0,4.000000e+01,2.0,0.706,3.0,2.0,1.0,2.0


# Save Data

In [ ]:
target_path = "Dataset/Data Versioning/Trained_V2-3.csv"
final = discretized_df.copy()

final.to_csv(os.path.join(drive_path, target_path))